In [ ]:
!pip install rank-bm25
!pip install datasets
!pip install chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install nltk
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
from datasets import load_dataset

# load the full ragbench dataset
ragbench = {}
for dataset in ['covidqa']: #, 'cuad', 'delucionqa']: #, 'emanual', 'expertqa', 'finqa', 'hagrid', 'hotpotqa', 'msmarco', 'pubmedqa', 'tatqa', 'techqa']:
  ragbench[dataset] = load_dataset("rungalileo/ragbench", dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/24.7k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/4.20M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/854k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/913k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1252 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/246 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/267 [00:00<?, ? examples/s]

In [ ]:
# New code - 12/4 10 pm

from nltk.tokenize import sent_tokenize
from transformers import AutoTokenizer

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("BAAI/LLM-Embedder")
#tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# Sliding window configuration
TOKEN_LIMIT = 512
SLIDING_WINDOW_OVERLAP = 100  # Overlap between consecutive chunks (in tokens)

# Function for chunking with token limit and sliding window
def chunk_with_token_limit(text, token_limit, overlap):
    sentences = sent_tokenize(text)  # Split text into sentences
    chunks = []  # Store resulting chunks
    current_chunk = []  # Temporarily hold sentences for the current chunk
    current_chunk_tokens = 0  # Token count for the current chunk

    for sentence in sentences:
        # Tokenize the sentence and calculate its token count
        sentence_tokens = tokenizer.tokenize(sentence)
        num_tokens = len(sentence_tokens)

        # print(f"Tokens: {sentence_tokens[0]}")

        # If adding this sentence exceeds the token limit
        if current_chunk_tokens + num_tokens > token_limit:
            # Save the current chunk
            chunk_text = " ".join(current_chunk)
            chunks.append(chunk_text)

            # Prepare the next chunk with overlap
            overlap_tokens = tokenizer.tokenize(" ".join(current_chunk[-1:]))
            current_chunk = [sentence for sentence in current_chunk[-(overlap // len(overlap_tokens)) :]] if current_chunk else []
            current_chunk_tokens = sum(len(tokenizer.tokenize(sent)) for sent in current_chunk)

        # Add the sentence to the current chunk
        current_chunk.append(sentence)
        current_chunk_tokens += num_tokens

    # Add the last chunk if it exists
    if current_chunk:
        chunk_text = " ".join(current_chunk)
        chunks.append(chunk_text)

    return chunks

# Function for processing the document and adding identifiers
def process_document_with_identifiers(document):
    processed_data = []
    title_count = 0

    for section in document:
        section_chunks = []
        passage_count = ord("a")  # Initialize passage letter as 'a'

        sentences = sent_tokenize(section)
        for sentence in sentences:
            if sentence.startswith("Title:"):
                identifier = f"{title_count}a"
                chunked_texts = chunk_with_token_limit(sentence, TOKEN_LIMIT, SLIDING_WINDOW_OVERLAP)
                for chunk in chunked_texts:
                    section_chunks.append([identifier, chunk])
                title_count += 1
                passage_count = ord("a")  # Reset passage count for the next title
            elif sentence.startswith("Passage:"):
                identifier = f"{title_count - 1}{chr(passage_count)}"
                chunked_texts = chunk_with_token_limit(sentence, TOKEN_LIMIT, SLIDING_WINDOW_OVERLAP)
                for chunk in chunked_texts:
                    section_chunks.append([identifier, chunk])
                passage_count += 1
            else:
                identifier = f"{title_count - 1}{chr(passage_count)}"
                chunked_texts = chunk_with_token_limit(sentence, TOKEN_LIMIT, SLIDING_WINDOW_OVERLAP)
                for chunk in chunked_texts:
                    section_chunks.append([identifier, chunk])
                passage_count += 1

        processed_data.append(section_chunks)
    return processed_data

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
# Code on 12/4, 10 pm

from datasets import load_dataset
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

'''
datasets = ['covidqa', 'cuad', 'delucionqa', 'emanual', 'expertqa',
           'finqa', 'hagrid', 'hotpotqa', 'msmarco', 'pubmedqa',
           'tatqa', 'techqa']
'''

# Insert embeddings into chromadb in batches as we are seeing memory issues when doing it at once.
datasets = ['covidqa'] #, 'cuad', 'delucionqa'] # List of dataset names

#datasets = ['covidqa', 'cuad', 'delucionqa', 'emanual']
#datasets = ['expertqa','finqa', 'hagrid', 'hotpotqa']
#datasets = ['msmarco', 'pubmedqa','tatqa', 'techqa']

# Initialize storage for documents, IDs, and metadata
all_documents = []
all_ids = []
all_metadatas = []


# Process each dataset
doc_idx = 0  # Global document index for unique IDs
for dataset in datasets:
    data = load_dataset("rungalileo/ragbench", dataset, split="train")

    for idx, row in tqdm(enumerate(data), desc=f"Processing {dataset}"):
        # Extract document text
        doc_text = row.get('documents', '')

        # Skip if no documents found
        if not doc_text:
            continue

        # Process the document
        processed_output = process_document_with_identifiers(doc_text)

        # Populate the lists
        for section_idx, section in enumerate(processed_output):
            for item_idx, (prefix, content) in enumerate(section):
                # Add the document
                document = f"[{prefix}] {content}"
                all_documents.append(document)

                # Construct a globally unique ID
                doc_id = f"{dataset}_{doc_idx}_{section_idx}_{item_idx}"
                all_ids.append(doc_id)

                # Construct metadata
                metadata = {
                    "dataset": dataset,
                    "global_index": doc_idx,
                    "section_index": section_idx,
                    "item_index": item_idx,
                    "prefix": prefix,
                    "type": "Title" if prefix.endswith("a") else "Passage",
                }
                all_metadatas.append(metadata)

        doc_idx += 1  # Increment global document index

# Step 4: Generate Embeddings
#embedder = SentenceTransformer("all-MiniLM-L6-v2")  # Pretrained sentence transformer
embedder = SentenceTransformer("BAAI/LLM-Embedder")  # Pretrained sentence transformer
batch_size = 2500  # Adjust based on available memory

# Generate embeddings in batches
all_embeddings = []
for i in tqdm(range(0, len(all_documents), batch_size), desc="Generating embeddings"):
    batch_docs = all_documents[i:i + batch_size]
    batch_embeddings = embedder.encode(batch_docs, show_progress_bar=True)
    all_embeddings.extend(batch_embeddings)


Processing covidqa: 1252it [00:06, 194.58it/s]


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/28.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Generating embeddings:  75%|███████▌  | 6/8 [38:33<12:42, 381.16s/it]

Batches:   0%|          | 0/79 [00:00<?, ?it/s]

Generating embeddings:  88%|████████▊ | 7/8 [44:34<06:14, 374.48s/it]

Batches:   0%|          | 0/41 [00:00<?, ?it/s]

Generating embeddings: 100%|██████████| 8/8 [47:48<00:00, 358.52s/it]


In [ ]:
from sentence_transformers import SentenceTransformer, util


def SimilarityScore(A2):
  # Initialize the model
  model = SentenceTransformer('all-MiniLM-L6-v2')
  # Text blocks A1 and A2

  # Combine the strings in A2 into a single string
  A2_combined = ' '.join(A2)

  # Generate embeddings for A1 and the combined A2
  embedding_A1 = model.encode(A1, convert_to_tensor=True)
  embedding_A2 = model.encode(A2_combined, convert_to_tensor=True)

  # Calculate cosine similarity
  similarity = util.cos_sim(embedding_A1, embedding_A2).item()

  return (f"Similarity: {similarity}")

In [ ]:
# Query
query = "How many clinical trials are registered?"
A1 = """
[ "Title: Global mapping of randomised trials related articles published in high-impact-factor medical journals: a cross-sectional analysis\nPassage: A total of 39,329 records were identified by the PubMed/MEDLINE search , and 39,305 articles met the study inclusion criteria after 24 records had been excluded . Table 2 details the general characteristics of the articles.", "Title: Global mapping of randomised trials related articles published in high-impact-factor medical journals: a cross-sectional analysis\nPassage: The number of articles increased exponentially over the period 1965-2017 . Approximately 60% of the articles have been published since 2000.", "Title: Global mapping of randomised trials related articles published in high-impact-factor medical journals: a cross-sectional analysis\nPassage: The evolution of RCTs has been an enduring and continuing process . Since the 1970s the publication landscape for RCTs has exhibited an exponential growth. For example, a 1965-2001 bibliometric analysis of the literature identified 369 articles published in 1970 compared to 11,159 published in 2000 . The development of clinical trial registries , the exponential increase in journals publishing trial protocols, results and secondary studies, and growing support for data-sharing policies have created an open research environment of transparency and accountability. Furthermore, the publication of reporting guidelines have served to facilitate the transition between research and reporting to ensure", "Title: Global mapping of randomised trials related articles published in high-impact-factor medical journals: a cross-sectional analysis\nPassage: 9, 2019." ]
"""


In [ ]:
from nltk.tokenize import word_tokenize

def Search_BM25(_query, t_documents, all_Docs):
  # Initialize BM25
  bm25 = BM25Okapi(t_documents)

  # Search and rank
  scores = bm25.get_scores(_query)
  top_docs = sorted(zip(scores, all_Docs), reverse=True)  # Sort by score
  ITR=1
  output_res=[]
  # Display results
  for score, doc in top_docs:
      res= f"Score: {score:.2f}, Document: {doc}"
      output_res.append(res)
      print(res)
      ITR+=1
      if ITR  > 10:
          break
  return output_res


In [ ]:

# Tokenize the documents
tokenized_documents = [word_tokenize(doc) for doc in all_documents]

tokenized_query = word_tokenize(query)

ot_re = Search_BM25(tokenized_query, tokenized_documents, all_documents)

SimilarityScore(ot_re)



Score: 23.47, Document: [3a] Among 23 trials found from the systematic review , there are nine clinical trials registered under the clinical trials registry for 2019-nCoV therapeutics .
Score: 21.45, Document: [1b] How many people become sick each year?
Score: 16.11, Document: [2b] How many additional mechanisms are we completely unaware of, and what sorts of vaccines could be produced to prevent emergences in humans of novel IAVs introduced by any such mechanism?
Score: 15.83, Document: [0c] In the US, at the moment over 500 clinical trials that focus on DNA vaccination are registered, targeting especially viral infections and cancer , while bacterial infections and autoimmune diseases are less of a topic.
Score: 15.21, Document: [3c] How will the act of consent be recorded ?
Score: 15.21, Document: [1b] How far back can we trace their contribution?
Score: 15.21, Document: [0b] How far back can we trace their contribution?
Score: 14.90, Document: [0c] How is condensin activated outsid

'Similarity: 0.4228084087371826'

In [ ]:

from nltk.tokenize import sent_tokenize, word_tokenize

# Tokenize the documents
tokenized_documents = all_documents

# Initialize BM25
bm25 = BM25Okapi(tokenized_documents)
tokenized_query = word_tokenize(query)

# Search and rank
scores = bm25.get_scores(tokenized_query)
top_docs = sorted(zip(scores, all_documents), reverse=True)  # Sort by score
ITR=1
output_res=[]
# Display results
for score, doc in top_docs:
    res= f"Score: {score:.2f}, Document: {doc}"
    output_res.append(res)
    print(res)
    ITR+=1
    if ITR  > 10:
        break



Score: 7.01, Document: [3a] Title: What we can do?
Score: 7.01, Document: [2a] Title: What we can do?
Score: 7.01, Document: [1a] Title: What we can do?
Score: 6.74, Document: [2a] Title: What Do We Have to Lose?
Score: 6.72, Document: [1a] Title: Two Birds with One Stone?
Score: 6.63, Document: [3d] The What was the research question?
Score: 6.55, Document: [2a] What information ought to be provided?
Score: 6.55, Document: [2a] Title: Do expert assessments converge?
Score: 6.55, Document: [2a] Title: Do expert assessments converge?
Score: 6.55, Document: [1b] How many people become sick each year?


In [ ]:
similarity_score=SimilarityScore(output_res)
print(similarity_score)

Similarity: 0.28507572412490845
